## Imports


In [4]:
import ipfshttpclient
import numpy as np
from pystac_client import Client
import rasterio
from rasterio.io import MemoryFile
import matplotlib.pyplot as plt

# Define IPFS client
# client = ipfshttpclient.connect()

## Fetch Landsat Scene from Washington D.C. Area


In [7]:
# Connect to STAC API and search for Landsat 9 imagery
catalog = Client.open("http://ec2-54-172-212-55.compute-1.amazonaws.com/api/v1/pgstac/")
bbox = [-76.964657, 38.978967, -76.928008, 39.002783]

search = catalog.search(
    collections=["landsat-c2l1"],
    bbox=bbox,
)

items = search.get_all_items()
len(items)


1

## Fetch bands 4 and 5 from the scene

In [11]:
item = items[0]
item
# # Get red and NIR band assets and access alternate keys
# red_band_cid = item.assets["red"].extra_fields["alternate"]["IPFS"]["href"]
# nir_band_cid = item.assets["nir"].extra_fields["alternate"]["IPFS"]["href"]
# print(f"Red band CID: {red_band_cid}")
# print(f"NIR band CID: {nir_band_cid}")

id: LC09_L1TP_015033_20221015_20221015_02_T1
"bbox: [-78.19032488487748, 37.84156096889577, -75.5633382854675, 39.951439673652615]"
datetime: 2022-10-15T15:46:53.348895Z
platform: LANDSAT_9
proj:epsg: 32618
"proj:shape: [7941, 7831]"
"instruments: ['OLI', 'TIRS']"
eo:cloud_cover: 0.05
"proj:transform: [30.0, 0.0, 218385.0, 0.0, -30.0, 4426515.0]"
view:off_nadir: 0
landsat:wrs_row: 033


## Download the bands from IPFS and pull them into memory

In [ ]:
# Download the bands from IPFS
def download_band(cid):
    stream = client.cat(cid)
    return np.asarray(stream)

red_band_stream = download_band(red_band_cid)
nir_band_stream = download_band(nir_band_cid)

# Read bands into numpy arrays using rasterio
with MemoryFile(red_band_stream) as memfile:
    with memfile.open() as src:
        red_band = src.read(1)

with MemoryFile(nir_band_stream) as memfile:
    with memfile.open() as src:
        nir_band = src.read(1)

## Calculate NDVI

In [ ]:
# Calculate NDVI
ndvi = np.where(
    (nir_band + red_band) == 0., # Avoid divide by zero errors
    0,
    (nir_band - red_band) / (nir_band + red_band)
)

## Plot NDVI

In [ ]:
# Plot NDVI histogram
plt.hist(ndvi.flatten(), bins=100)
plt.title('NDVI Histogram')
plt.xlabel('NDVI Value')
plt.ylabel('Pixel Count')
plt.show()